# GSM8K Test Metrics.ipynb

Run all metrics on the test conversation.


In [43]:
# Load test_conversation.txt
with open('test_conversation.txt', 'r') as f:
    test_conversation = f.readlines()

# Print with each list item on a new line
for line in test_conversation:
    print(line)


Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

Task: Generate 3 different possible one-step calculations to serve as step 1 in solving the problem. Only work on step 1. Put each calculation on a new line. Do not number them.

Add 16 and 4 and multiply by 2 

Subtract 3 from 16 and multiply by 2 

Multiply 16 and 2 and subtract 7

Task: State the calculation above that is most likely to contribute to solving the problem. If it fully solves the original problem, also output STOP and the solution to the problem. If none of the calculations are correct, output ERROR and generate three new ones.

Subtract 3 from 16 and multiply by 2 - STOP - 28 x 2 = $56


In [44]:
# Load answer for this question

import json
import regex as re

# Load the test question from test.jsonl
with open('test.jsonl', 'r') as f:
    test = f.readlines()

# Strip items between << and >> in each line
test = [re.sub(r'<<.*?>>', '', line) for line in test]

#print(test)
print(json.loads(test[0])['answer'])


Janet sells 16 - 3 - 4 = 9 duck eggs a day.
She makes 9 * 2 = $18 every day at the farmer’s market.
#### 18


## Accuracy

Counts as zero points, model is incorrect.

## Length

It's important to use the OpenAI tiktoken tokenizer.

In [45]:
# Concatenate conversation

concat_convo = ''.join(test_conversation)

concat_convo


"Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\nTask: Generate 3 different possible one-step calculations to serve as step 1 in solving the problem. Only work on step 1. Put each calculation on a new line. Do not number them.\nAdd 16 and 4 and multiply by 2 \nSubtract 3 from 16 and multiply by 2 \nMultiply 16 and 2 and subtract 7\nTask: State the calculation above that is most likely to contribute to solving the problem. If it fully solves the original problem, also output STOP and the solution to the problem. If none of the calculations are correct, output ERROR and generate three new ones.\nSubtract 3 from 16 and multiply by 2 - STOP - 28 x 2 = $56"

In [46]:
import tiktoken

enc = tiktoken.encoding_for_model("text-davinci-003")

print(enc.encode("hello world"))
print(enc.encode(concat_convo))


[31373, 995]
[48, 25, 28111, 447, 247, 82, 39694, 3830, 1467, 9653, 583, 1110, 13, 1375, 25365, 1115, 329, 12607, 790, 3329, 290, 275, 1124, 27563, 1040, 329, 607, 2460, 790, 1110, 351, 1440, 13, 1375, 16015, 262, 17675, 379, 262, 9818, 6, 1910, 4445, 329, 720, 17, 583, 4713, 22045, 5935, 13, 1374, 881, 287, 5054, 857, 673, 787, 790, 1110, 379, 262, 9818, 6, 1910, 30, 198, 25714, 25, 2980, 378, 513, 1180, 1744, 530, 12, 9662, 16765, 284, 4691, 355, 2239, 352, 287, 18120, 262, 1917, 13, 5514, 670, 319, 2239, 352, 13, 5930, 1123, 17952, 319, 257, 649, 1627, 13, 2141, 407, 1271, 606, 13, 198, 4550, 1467, 290, 604, 290, 29162, 416, 362, 220, 198, 7004, 83, 974, 513, 422, 1467, 290, 29162, 416, 362, 220, 198, 15205, 541, 306, 1467, 290, 362, 290, 34128, 767, 198, 25714, 25, 1812, 262, 17952, 2029, 326, 318, 749, 1884, 284, 8676, 284, 18120, 262, 1917, 13, 1002, 340, 3938, 39107, 262, 2656, 1917, 11, 635, 5072, 44934, 290, 262, 4610, 284, 262, 1917, 13, 1002, 4844, 286, 262, 16765, 389, 3376

In [47]:
print(len(enc.encode(concat_convo)))


211


## Cost

2 cents per 1000 tokens

In [48]:
print("$" + str(len(enc.encode(concat_convo)) * (1 / 1000) * 0.02))


$0.00422


## Length Relative to Baseline

Relative to provided question and answer.

In [49]:
provided_question_answer = json.loads(test[0])['question'] + "\n" + json.loads(test[0])['answer']
print(provided_question_answer)

print('ratio')
print(len(enc.encode(concat_convo)) / len(enc.encode(provided_question_answer)))


Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Janet sells 16 - 3 - 4 = 9 duck eggs a day.
She makes 9 * 2 = $18 every day at the farmer’s market.
#### 18
ratio
2.0485436893203883


Relative to direct prompting

In [50]:
# Read in direct prompting file
with open('direct_prompting_baseline.txt', 'r') as f:
    direct_prompting_baseline = f.readlines()

print(direct_prompting_baseline)

# Direct prompting answer is the last line
direct_prompting_output = direct_prompting_baseline[-1]
print(direct_prompting_output)


["Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n", 'A:\n', "Janet makes $8 per day at the farmers' market. She sells 9 eggs (16 eggs - 3 eggs breakfast - 4 eggs baked) at $2 per egg, for a total of $18."]
Janet makes $8 per day at the farmers' market. She sells 9 eggs (16 eggs - 3 eggs breakfast - 4 eggs baked) at $2 per egg, for a total of $18.


In [51]:
# Concatenate direct prompting
concat_dp = ''.join(direct_prompting_baseline)


In [52]:
print(concat_convo)
print(concat_dp)

print('ratio')
print(len(enc.encode(concat_convo)) / len(enc.encode(concat_dp)))


Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Task: Generate 3 different possible one-step calculations to serve as step 1 in solving the problem. Only work on step 1. Put each calculation on a new line. Do not number them.
Add 16 and 4 and multiply by 2 
Subtract 3 from 16 and multiply by 2 
Multiply 16 and 2 and subtract 7
Task: State the calculation above that is most likely to contribute to solving the problem. If it fully solves the original problem, also output STOP and the solution to the problem. If none of the calculations are correct, output ERROR and generate three new ones.
Subtract 3 from 16 and multiply by 2 - STOP - 28 x 2 = $56
Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her fri

## Accuracy Gains Per Increase in Tokens

Direct prompting is correct and tree of toughts is not

In [53]:
print(len(enc.encode(concat_convo)) - len(enc.encode(concat_dp)))

print('gains per')
print((-100) / (len(enc.encode(concat_convo)) - len(enc.encode(concat_dp))))


99
gains per
-1.0101010101010102


## Number of Reasoning Steps

Check linebreaks and periods skipping decimal places.

In [54]:
# We need to get a concatenated version of just the output of the model
output_lines = [line for line in test_conversation if not line.startswith('Q:') and not line.startswith('Task:')]
print(output_lines)

input_lines = [line for line in test_conversation if line not in output_lines]

concat_output = ''.join(output_lines)
concat_input = ''.join(input_lines)


['Add 16 and 4 and multiply by 2 \n', 'Subtract 3 from 16 and multiply by 2 \n', 'Multiply 16 and 2 and subtract 7\n', 'Subtract 3 from 16 and multiply by 2 - STOP - 28 x 2 = $56']


In [55]:
# import nltk

# test_sentences = nltk.sent_tokenize("I ate a pie. It was great! I want more.")
# print(test_sentences)


In [56]:
print(concat_output)


Add 16 and 4 and multiply by 2 
Subtract 3 from 16 and multiply by 2 
Multiply 16 and 2 and subtract 7
Subtract 3 from 16 and multiply by 2 - STOP - 28 x 2 = $56


In [57]:
print('num linebreaks')
print(concat_output.count('\n'))

print('num periods - require a space after the period. add one for the last sentence')
print(concat_output.count('. ') + 1)


num linebreaks
3
num periods - require a space after the period. add one for the last sentence
1


## Contrasting Words

Search for {'while', 'but', 'though', 'although', 'other', 'others', 'however'}.

In [58]:
print('num contrasting words')

for word in ['while', 'but', 'though', 'although', 'other', 'others', 'however']:
    print(word)
    print(concat_output.count(word))


num contrasting words
while
0
but
0
though
0
although
0
other
0
others
0
however
0


## Differences in responses versus prompts

In [59]:
# Number of reasoning steps

print('linebreaks')
print(concat_output.count('\n') - concat_input.count('\n'))
print('periods')
print(concat_output.count('. ') - concat_input.count('. '))


linebreaks
0
periods
-8


In [60]:
# Contrasting words

for word in ['while', 'but', 'though', 'although', 'other', 'others', 'however']:
    print(word)
    print(concat_output.count(word) - concat_input.count(word))


while
0
but
-1
though
0
although
0
other
0
others
0
however
0


## Differences versus direct baseline

In output, for concat_output versus concat_dp_output

In [61]:
concat_dp_output = ''.join(direct_prompting_output)
print(concat_dp_output)


Janet makes $8 per day at the farmers' market. She sells 9 eggs (16 eggs - 3 eggs breakfast - 4 eggs baked) at $2 per egg, for a total of $18.


In [62]:
# Number of reasoning steps

print('linebreaks')
print(concat_output.count('\n') - concat_dp_output.count('\n'))
print('periods')
print(concat_output.count('. ') - concat_dp_output.count('. '))


linebreaks
3
periods
-1


In [63]:
# Contrasting words

for word in ['while', 'but', 'though', 'although', 'other', 'others', 'however']:
    print(word)
    print(concat_output.count(word) - concat_dp_output.count(word))


while
0
but
0
though
0
although
0
other
0
others
0
however
0


## Differences versus Provided Answer Baseline

In output, for concat_output versus json.loads(test[0])['answer']

In [64]:
provided_answer = json.loads(test[0])['answer']
print(provided_answer)


Janet sells 16 - 3 - 4 = 9 duck eggs a day.
She makes 9 * 2 = $18 every day at the farmer’s market.
#### 18


In [65]:
print(concat_output)


Add 16 and 4 and multiply by 2 
Subtract 3 from 16 and multiply by 2 
Multiply 16 and 2 and subtract 7
Subtract 3 from 16 and multiply by 2 - STOP - 28 x 2 = $56


In [66]:
# Number of reasoning steps

print('linebreaks')
print(concat_output.count('\n') - provided_answer.count('\n'))

print('periods')
print(concat_output.count('. ') - provided_answer.count('. '))


linebreaks
1
periods
0


In [67]:
# Contrasting words

for word in ['while', 'but', 'though', 'although', 'other', 'others', 'however']:
    print(word)
    print(concat_output.count(word) - provided_answer.count(word))


while
0
but
0
though
0
although
0
other
0
others
0
however
0
